# Problem Set 1 - Neural network implementation

Team number: PS 1 B

Team members: 
- Giorgio Coppola (224545)
- Minho Kang (239742)
- Sofiya Berdiyeva (246934)

As described in section "2 Neural network implementation" of assignment 1, the goal is to build a shallow neural network from scratch using different approaches. To validate that your code is working and that the network is actually learning something, please use the following MNIST classification task. Finally, please submit proof of the learning progress as described in the assignment.

## Imports

In [5]:
import random
import pandas as pd
import numpy as np
from sklearn import model_selection
import sklearn.datasets as sk_datasets
import torchvision.datasets as torch_datasets
from torchvision import transforms
import torch
import matplotlib.pyplot as plt

from scratch.network import Network
from scratch.res_network import ResNetwork
from pytorch.network import TorchNetwork
from scratch.utils import *

In [6]:
# Automatically load changes in imported modules
%load_ext autoreload
%autoreload 2

# Explicitly set seed for reproducibility
GLOBAL_RANDOM_STATE = 42

random.seed(GLOBAL_RANDOM_STATE)
np.random.seed(GLOBAL_RANDOM_STATE)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## A) Neural Network Classifier from Scratch

### Data

In [7]:
# Download MNIST dataset
x, y_cat = sk_datasets.fetch_openml('mnist_784', version=1, return_X_y=True, cache=True, as_frame=False)

# Preprocess dataset
x = (x / 255).astype('float32')
y_cat = y_cat.astype(int)
# One-hot encode y
y = np.zeros((len(y_cat), 10))
for i, val in enumerate(y_cat):
    y[i, val] = 1

# Use only small subset of data for faster training
x = x[:1000]
y = y[:1000]

# Split data into train and validation set
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2, random_state=GLOBAL_RANDOM_STATE)



### ML Model & Training

In [8]:
fnn = Network(sizes=[784, 128, 64, 10], learning_rate=0.1, epochs=50)
fnn.fit(x_train, y_train, x_val, y_val, cosine_annealing_lr=False)

Epoch: 1, Training Time: 0.00s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 2, Training Time: 0.01s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 3, Training Time: 0.01s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 4, Training Time: 0.01s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 5, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 6, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 7, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 8, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 9, Training Time: 0.03s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 10, Training Time: 0.03s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 11, Training Time: 0.03s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 12, Training Time: 0.04s, Training Accuracy: 0.00%, Vali

### Test cosine annealing scheduler

In [9]:
fnn.fit(x_train, y_train, x_val, y_val, cosine_annealing_lr=True)

Epoch: 1, Training Time: 0.01s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 2, Training Time: 0.01s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 3, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 4, Training Time: 0.02s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 5, Training Time: 0.03s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 6, Training Time: 0.03s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 7, Training Time: 0.04s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 8, Training Time: 0.04s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 9, Training Time: 0.05s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 10, Training Time: 0.05s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 11, Training Time: 0.05s, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch: 12, Training Time: 0.06s, Training Accuracy: 0.00%, Vali

### Test residual neural network

In [10]:
res_nn = ResNetwork(sizes=[784, 128, 128, 10], learning_rate=1, epochs=50)
res_nn.fit(x_train, y_train, x_val, y_val)

AttributeError: 'ResNetwork' object has no attribute 'epochs'

## B) Neural Network Classifier using Torch

### Data

In [13]:
# Define data preprocessing steps
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])

# Download MNIST dataset
train_set = torch_datasets.MNIST('data', train=True, download=True, transform=transform)
val_set = torch_datasets.MNIST('data', train=False, download=True, transform=transform)

# Use only small subset of data for faster training
train_set = torch.utils.data.Subset(train_set, range(1000))
val_set = torch.utils.data.Subset(val_set, range(1000))

# Utilize PyTorch DataLoader from simplified & harmonized loading of data
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=1)


### ML Model & Training

In [14]:
torch_nn = TorchNetwork(sizes=[784, 128, 64, 10], learning_rate=0.2, epochs=50, random_state=GLOBAL_RANDOM_STATE)
torch_nn.fit(train_loader, val_loader)

TypeError: eq() received an invalid combination of arguments - got (NoneType, Tensor), but expected one of:
 * (Tensor input, Tensor other, *, Tensor out = None)
 * (Tensor input, Number other, *, Tensor out = None)


## C) Visualize accuracy & hyperparameter tuning

Here, you should compare the accuracy of all trained models. Optionally, you can also show the results of hyperparameter tuning and comment which hyperparameters work best for this task.

In [ ]:
def accuracy_plot(model):
    plt.plot(model.train_accuracies, label='Training Accuracy') #Need to implement in each model
    plt.plot(model.val_accuracies, label='Validation Accuracy') #Need to implement in each model
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy over Epochs')
    plt.legend()
    plt.show()

In [ ]:
accuracy_plot(fnn)
accuracy_plot(res_nn)
accuracy_plot(torch_nn)
